In [ ]:
!pip install nltk seaborn matplotlib sentence_transformers scikit-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached sentence-transformers-2.2.2.tar.gz (85 kB)
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 58.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 47.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 60.2 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125942 sha256=78568ec0625155d8ef5039a8cbf76da3a291b9f8c096456662e76366c197da06
  Stored in directory: /root/.cache/pip/wheels/71/67/06/162a3760c40d74dd40bc855d527008d26341c2b0ecf3e8e11f
Successfully built sentence_transformers


In [ ]:
import numpy as np
import pandas as pd
import re
import matplotlib
import nltk
import seaborn as sns
from nltk.tokenize import word_tokenize
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.ticker as ticker 
import matplotlib.pyplot as plt 

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
plt.style.use('ggplot')

In [ ]:
def clean_text_BERT(text):

    # Convert words to lower case.
    text = text.lower()

    # Remove special characters and numbers. This also removes the dates 
    # which are not important in classifying expenses
    text = re.sub(r'[^\w\s]|https?://\S+|www\.\S+|https?:/\S+|[^\x00-\x7F]+|\d+', '', str(text).strip())
  
    # Tokenise 
    text_list = word_tokenize(text)
    result = ' '.join(text_list)
    return result

In [ ]:
def change_categories(text):
  text = text.replace("Government + Non-Profit", "Services")
  text = text.replace("General Merchandise", "Shopping")
  text = text.replace("Bank Transfers", "Transfers")
  text = text.replace("General Services", "Services")
  return text

In [ ]:
df = pd.read_csv("data.csv")

In [ ]:
df

,worker_id,task_id,task_response_id,transaction_text,transaction_value,transaction_type,transaction_description,Select the best category for this bank transaction
0,MFVVAY4H7Y4H,564ec75d-8299-414e-813c-f61b60f83feb,78b46640-f5d8-4df1-a57e-2e88c73aa90f,UBER TRIP 8005928996 /CA US CARD PURCHASE,11.8,Debit,Uber trip from the hospital to my house.,Transportation / Public Transport
1,J6TM2D4MANTG,4be7ef74-e6d6-4ab6-ac4a-bb0ac1378afb,783889f8-968c-4a03-8086-0c5cf8b70f03,DEBIT CARD PURCHASE XXXXX8046 LYFT *2 RIDES 04...,20.64,Debit,Rideshare charges for two rides.,"Rent & Utilities / Electricity, natural gas an..."
2,YYCA2DXN94NG,59c515a8-c95e-4d19-a0b6-e91eeec47d47,42b5d790-6c8b-44d9-bd1d-aa172f927a4d,8046 Debit Card Purchase Uber Help.Uber.Ca,4,Debit,This is a tip paid for an Uber ride.,Transportation / Taxis and ride shares
3,J6TM2D4MANTG,4be7ef74-e6d6-4ab6-ac4a-bb0ac1378afb,07e8e9e0-d0ce-4ea4-914f-7a176007d8c9,DEBIT CARD PURCHASE XXXXX8046 LYFT *2 RIDES 04...,20.64,Debit,Rideshare charges for two rides.,Transportation / Taxis and ride shares
4,YYCA2DXN94NG,e164c347-f85b-441f-b2d6-150fc4c0aca3,790681e8-70d6-4d4e-97c9-5c922e6f9676,ENTERPRISE RENT A CAR E EGHAM,91.24,Debit,Car rental charge.,Travel / Rental cars
...,...,...,...,...,...,...,...,...
517,YYCA2DXN94NG,6a939e3a-e221-4140-a4ae-44df66a090da,18d92eed-3aa3-4c79-bf4e-1797fc7d9f0f,ATM TRANSACTION FEE REIMBURSEMENT,3,Credit,Fees reimbursed.,Bank Transfers / ATM fees
518,MFVVAY4H7Y4H,b1ae3c34-adc7-4e2c-abad-3cc591185583,00708885-aec2-4db1-81ca-676ae03b7f7e,ATM Fee Reimbursement,10,Credit,Reimbursement from Ally for ATM fees from othe...,Bank Transfers / ATM fees
519,MQGN7N3PKYRA,b9bb5a26-94f9-43b7-813f-5f2b22ff2358,47713ff7-739e-425d-9ed2-27346e2e9318,ATM WITHDRAWAL FEE,3,Debit,ATM fees.,Bank Transfers / ATM fees
520,J6TM2D4MANTG,ad1b61d8-bd20-4c5f-bf9b-0f8fd79a6a76,c1b8292e-cb63-44ec-a373-25bf3240a45f,Withdrwl Kraljevo Filijala,5,Debit,ATM fees.,Bank Transfers / ATM fees


In [ ]:
df = df.drop(columns=['worker_id', 'task_id', 'task_response_id', 'transaction_description'], axis=1)

In [ ]:
df.columns

Index(['transaction_text', 'transaction_value', 'transaction_type',
       'Select the best category for this bank transaction '],
      dtype='object')

In [ ]:
df.rename(columns={df.columns.values[3]: "transaction_category"}, inplace=True)

In [ ]:
df.columns

Index(['transaction_text', 'transaction_value', 'transaction_type',
       'transaction_category'],
      dtype='object')

In [ ]:
df["transaction_category"] = df["transaction_category"].str.split("/")

In [ ]:
df["transaction_category"] = df["transaction_category"].str[0]

In [ ]:
df.dropna(inplace=True, subset=["transaction_category"])

In [ ]:
df["transaction_category"].apply(change_categories)

0        Transportation 
1      Rent & Utilities 
2        Transportation 
3        Transportation 
4                Travel 
             ...        
517           Transfers 
518           Transfers 
519           Transfers 
520           Transfers 
521           Transfers 
Name: transaction_category, Length: 521, dtype: object

In [ ]:
df

,transaction_text,transaction_value,transaction_type,transaction_category
0,UBER TRIP 8005928996 /CA US CARD PURCHASE,11.8,Debit,Transportation
1,DEBIT CARD PURCHASE XXXXX8046 LYFT *2 RIDES 04...,20.64,Debit,Rent & Utilities
2,8046 Debit Card Purchase Uber Help.Uber.Ca,4,Debit,Transportation
3,DEBIT CARD PURCHASE XXXXX8046 LYFT *2 RIDES 04...,20.64,Debit,Transportation
4,ENTERPRISE RENT A CAR E EGHAM,91.24,Debit,Travel
...,...,...,...,...
517,ATM TRANSACTION FEE REIMBURSEMENT,3,Credit,Bank Transfers
518,ATM Fee Reimbursement,10,Credit,Bank Transfers
519,ATM WITHDRAWAL FEE,3,Debit,Bank Transfers
520,Withdrwl Kraljevo Filijala,5,Debit,Bank Transfers


In [ ]:
text_raw = df["transaction_text"]
text_BERT = text_raw.apply(lambda x: clean_text_BERT(x))

In [ ]:
bert_input = text_BERT.tolist()
model = SentenceTransformer('paraphrase-mpnet-base-v2') 
embeddings = model.encode(bert_input, show_progress_bar = True)
embedding_BERT = np.array(embeddings)

Batches:   0%|          | 0/17 [00:00<?, ?it/s]

In [ ]:
df_embedding = pd.DataFrame(embeddings)
df_embedding.head()

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
0,-0.062850,0.106256,0.019853,-0.044087,0.103625,-0.080157,0.025014,-0.098283,0.117487,-0.003308,...,0.089001,-0.039998,-0.011559,0.173814,-0.064049,0.183582,-0.046155,-0.030116,-0.123995,-0.032116
1,-0.009699,0.075544,0.193329,0.056223,0.016654,-0.061779,0.030519,0.018068,0.080875,-0.012493,...,0.065021,0.034326,-0.060993,0.127640,-0.024639,0.188547,-0.091153,0.037108,-0.118958,-0.028447
2,0.017341,-0.117546,0.052712,0.073000,0.137966,-0.087853,0.118744,-0.122304,-0.015431,-0.010141,...,0.080639,0.070114,0.091995,0.089350,-0.094082,0.079633,-0.085679,-0.023610,-0.024619,-0.004216
3,-0.009699,0.075544,0.193329,0.056223,0.016654,-0.061779,0.030519,0.018068,0.080875,-0.012493,...,0.065021,0.034326,-0.060993,0.127640,-0.024639,0.188547,-0.091153,0.037108,-0.118958,-0.028447
4,-0.141413,-0.103217,0.014222,0.105725,0.112086,-0.007109,0.194224,-0.008600,0.185192,-0.109939,...,0.055898,-0.066721,0.294194,0.157080,-0.155761,0.354455,-0.170868,-0.116015,-0.143928,-0.198730


In [ ]:
test_raw_df = pd.read_csv("1.csv")

In [ ]:
test_raw_df = test_raw_df["Narrative"]

In [ ]:
test_raw_BERT = test_raw_df.apply(clean_text_BERT)

In [ ]:
bert_input_test = test_raw_BERT.tolist()
embeddings_test = model.encode(bert_input_test, show_progress_bar = True)
embedding_BERT_test = np.array(embeddings_test)
df_embedding_bert_test = pd.DataFrame(embeddings_test)


Batches:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
# Find the most similar word embedding with unseen data in the training data

similarity_new_data = cosine_similarity(embedding_BERT_test, embedding_BERT)
similarity_df = pd.DataFrame(similarity_new_data)

# Returns index for most similar embedding
# See first column of the output dataframe below
index_similarity = similarity_df.idxmax(axis = 1)

# Return dataframe for most similar embedding/transactions in training dataframe
data_inspect = df.iloc[index_similarity, :].reset_index(drop = True)

unseen_verbatim = test_raw_df
matched_verbatim = data_inspect['transaction_text']
annotation = data_inspect['transaction_category']

d_output = pd.DataFrame({
            'unseen_transaction': unseen_verbatim,
            'matched_transaction': matched_verbatim, 
            'matched_class': annotation
            
            })

In [ ]:
d_output

,unseen_transaction,matched_transaction,matched_class
0,Online Transfer FT2218448S77\BNK - Diet subscr...,Internet transfer from Online Savings account ...,Bank Transfers
1,Online Transfer FT22186BQPSY\BNK - Transfer to...,FCB FUNDS TRANSFER FROM X3428,Bank Transfers
2,ATM WITHDRAWAL FT22186917LY\BNK CIB MOBILE EL ...,ATM WITHDRAWAL FEE,Bank Transfers
3,POS PURCHASE FT22187NLGGK\BNK PADDLE NET TRIPM...,POS Purchase Banh Mi & Ti Pittsburgh PA,Food & Drink
4,POS PURCHASE FT22188C286L\BNK PAYMOB RABBITSCO...,POS Purchase Banh Mi & Ti Pittsburgh PA,Food & Drink
...,...,...,...
85,POS PURCHASE FT22262XTYGF\BNK SPOTIFY STOCKHOL...,Spotify,Entertainment
86,POS PURCHASE FT22263ZWC26\BNK PAYPAL ZALANDOSE...,8382 Debit Card Purchase Kfteausa_247,Food & Drink
87,POS PURCHASE FT22263QVGPV\BNK Talabat Cairo EG...,POS Purchase Banh Mi & Ti Pittsburgh PA,Food & Drink
88,Online Transfer FT22272TW0HV\BNK - Transfer to...,Internet transfer from Online Savings account ...,Bank Transfers
